In [ ]:
import torch
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
from Dataset import Dataset, ELM, device
torch.set_default_dtype(torch.float64)

In [ ]:
x_data = np.float64(np.linspace(-15, 15, 600))
y_data = np.float64(np.sin(x_data) / x_data)
# y_data = np.float64(np.sin(x_data) / x_data + np.random.normal(0, 0.1, 600))

In [ ]:
dataset = Dataset(x_data, y_data)
trainset, testset = torch.utils.data.random_split(dataset, [0.9, 0.1], generator=torch.Generator().manual_seed(42))

activation = nn.Sigmoid()

In [ ]:
Lmax, epsilon = 1000, 1e-7
E, L = trainset[:][1], 0
k = 5 # 1 dla I-ELM, > 1 dla EI-ELM
temp_model = ELM(1, Lmax, 1).to(device)
Error = [torch.norm(E)]

while L < Lmax and torch.norm(E) > epsilon:
    L += 1
    Enorm = torch.norm(E)
    for i in range(k):
        W1 = torch.randn(2, 1).to(device)
        h = activation(W1[0].reshape(-1, 1) + W1[1] @ trainset[:][0].reshape(1, -1).to(device))
        beta = torch.div(E.reshape(1, -1).to(device) @ h.T, h @ h.T).to(device)
        if Enorm > torch.norm(E.to(device) - beta @ h) or L == 1:
            h_star = h.to(device)
            W1_star = W1.to(device)
            beta_star = beta.to(device)
            Enorm = torch.norm(E.to(device) - beta @ h)
    E = E.to(device) - beta_star @ h_star
    Error.append(torch.norm(E.cpu()))
    temp_model.fc1.bias.data[L-1], temp_model.fc1.weight.data[L-1], temp_model.fc2.weight.data[:,L-1] = W1_star[0], W1_star[1], beta_star

plt.plot(Error)
plt.plot([epsilon]*len(Error), 'r--')
plt.grid()
plt.show()

In [ ]:
model = ELM(1, L, 1).to(device)

model.fc1.bias.data, model.fc1.weight.data, model.fc2.weight.data = \
temp_model.fc1.bias.data[:L], temp_model.fc1.weight.data[:L], temp_model.fc2.weight.data[:L]

In [ ]:
print(f'Train error: {(model(trainset[:][0].clone().detach().reshape(-1, 1).to(device))-trainset[:][1].reshape(-1, 1).to(device)).reshape(-1).pow(2).mean()}')
print(f'Test error: {(model(testset[:][0].clone().detach().reshape(-1, 1).to(device))-testset[:][1].reshape(-1, 1).to(device)).reshape(-1).pow(2).mean()}')
plt.plot(x_data, y_data)
plt.plot(x_data, model(torch.tensor(x_data).reshape(-1, 1).to(device)).cpu().detach().numpy(), 'r--')
plt.legend(['Dane', 'Model'], loc='upper right')
plt.xlabel('x')
plt.ylabel('y')
plt.grid()
plt.show()

In [ ]:
plt.plot([np.min(y_data), np.max(y_data)], [np.min(y_data), np.max(y_data)])
plt.plot(y_data, model(torch.tensor(x_data).reshape(-1, 1)).detach().numpy(), '.', markersize=1)
plt.grid()
plt.show()